In [24]:
import numpy as np
from PIL import Image
import time

In [ ]:
def time_counting_and_save(fn, *args, **kwargs):
    start_time = time.time()
    scaled_array = fn(*args, **kwargs)
    end_time = time.time()
    print(f"Function '{fn.__name__}' 耗时：{end_time - start_time:.4f}秒")

    # 转换回PIL图像并保存
    img_scaled = Image.fromarray(scaled_array)
    # 保存缩放后的图片
    img_scaled.save(f'{fn.__name__}_scaled_image.JPG')
    return scaled_array

# 矩阵版最近邻插值, 比较慢
def nearest_neighbor_scaling_by_matrix(img_array, scale_x, scale_y):
    original_h, original_w = img_array.shape[:2]
    new_h = int(original_h * scale_y)
    new_w = int(original_w * scale_x)
    scaled_array = np.zeros((new_h, new_w, img_array.shape[2]), dtype=img_array.dtype)
    scale_matrix_inv = np.array([
        [1/scale_x, 0], 
        [0, 1/scale_y]
    ])
    xy = np.array([0, 0])

    for i in range(new_h):
        for j in range(new_w):
            # 计算在原图像中对应的坐标
            xy[0] = i
            xy[1] = j
            orig_i, orig_j = np.dot(scale_matrix_inv, xy).astype(int)

            # 确保坐标不超出边界
            orig_i = min(orig_i, original_h - 1)
            orig_j = min(orig_j, original_w - 1)
            
            # 复制像素值
            scaled_array[i, j] = img_array[orig_i, orig_j]
    return scaled_array

# 矩阵版最近邻插值，不使用矩阵乘法，稍微快一点
def nearest_neighbor_scaling(img_array, scale_x, scale_y):
    original_h, original_w = img_array.shape[:2]
    new_h = int(original_h * scale_y)
    new_w = int(original_w * scale_x)
    scaled_array = np.zeros((new_h, new_w, img_array.shape[2]), dtype=img_array.dtype)

    for i in range(new_h):
        for j in range(new_w):
            # 计算在原图像中对应的坐标
            orig_i = int(i / scale_y)
            orig_j = int(j / scale_x)

            # 确保坐标不超出边界
            orig_i = min(orig_i, original_h - 1)
            orig_j = min(orig_j, original_w - 1)
            
            # 复制像素值
            scaled_array[i, j] = img_array[orig_i, orig_j]
    return scaled_array

# 优化版本：向量化计算,两次向量化，高效计算
def nearest_neighbor_scaling_optimized(img_array, scale_x, scale_y):
    original_h, original_w = img_array.shape[:2]
    new_h, new_w = int(original_h * scale_y), int(original_w * scale_x)
    
    # 一次性创建所有坐标
    j_indices, i_indices = np.meshgrid(np.arange(new_w), np.arange(new_h))
    
    # 向量化的逆变换计算
    orig_i = (i_indices / scale_y).astype(int)
    orig_j = (j_indices / scale_x).astype(int)
    
    # 边界裁剪
    orig_i = np.clip(orig_i, 0, original_h - 1)
    orig_j = np.clip(orig_j, 0, original_w - 1)
    
    # 向量化索引
    scaled_array = img_array[orig_i, orig_j]
    
    return scaled_array

def double_linear_scaling(img_array, scale_x, scale_y):
        
    original_h, original_w = img_array.shape[:2]
    new_h, new_w = int(original_h * scale_y), int(original_w * scale_x)

    # 创建新的空图像数组
    if len(img_array.shape) == 3:  # 彩色图像
        scaled_array = np.zeros((new_h, new_w, img_array.shape[2]), dtype=img_array.dtype)
    else:  # 灰度图像
        scaled_array = np.zeros((new_h, new_w), dtype=img_array.dtype)

    for i in range(new_h):
        for j in range(new_w):
            # 计算在原图像中对应的坐标
            orig_i = i / scale_y
            orig_j = j / scale_x

            # 计算插值
            x_l = int(orig_j)
            x_h = min(x_l + 1, original_w - 1)
            y_l = int(orig_i)
            y_h = min(y_l + 1, original_h - 1)

            # 进行双线性插值
            """
                (y_l, x_l) - r1 - (y_l, x_h)
                |            |             | 
                |----------- p ------------|
                |            |             |
                (y_h, x_l) - r2 - (y_h, x_h)
            """
            if x_l == x_h or y_l == y_h:
                scaled_array[i, j] = img_array[y_l, x_l]
            else:
                r1 = (img_array[y_l, x_l] * (x_h - orig_j) + img_array[y_l, x_h] * (orig_j - x_l)) / (x_h - x_l)
                r2 = (img_array[y_h, x_l] * (x_h - orig_j) + img_array[y_h, x_h] * (orig_j - x_l)) / (x_h - x_l)
                scaled_array[i, j] = ((r1 * (y_h - orig_i) + r2 * (orig_i - y_l)) / (y_h - y_l)).astype(img_array.dtype)

    return scaled_array

In [51]:
# 加载图片并转换为numpy数组
img = Image.open('pixel.png')
img_array = np.array(img)

# 设置缩放因子
scale_x = 2  # 水平方向放大2倍
scale_y = 2  # 垂直方向放大2倍

scale_matrix = np.array([
    [scale_x, 0], 
    [0, scale_y]
])
scale_matrix_inv = np.linalg.inv(scale_matrix)

# 获取原始图像尺寸
original_h, original_w = img_array.shape[:2]

# 计算新的尺寸
new_h = int(original_h * scale_y)
new_w = int(original_w * scale_x)

# 创建新的空图像数组
if len(img_array.shape) == 3:  # 彩色图像
    scaled_array = np.zeros((new_h, new_w, img_array.shape[2]), dtype=img_array.dtype)
else:  # 灰度图像
    scaled_array = np.zeros((new_h, new_w), dtype=img_array.dtype)


scaled_array = time_counting_and_save(nearest_neighbor_scaling_by_matrix, img_array, scale_x, scale_y)
scaled_array = time_counting_and_save(nearest_neighbor_scaling,img_array, scale_x, scale_y)
scaled_array = time_counting_and_save(nearest_neighbor_scaling_optimized, img_array, scale_x, scale_y)
scaled_array = time_counting_and_save(double_linear_scaling, img_array, scale_x, scale_y)
scaled_array = time_counting_and_save(double_linear, img_array, scale_x, scale_y)


Function 'nearest_neighbor_scaling_by_matrix' 耗时：0.1350秒
Function 'nearest_neighbor_scaling' 耗时：0.0289秒
Function 'nearest_neighbor_scaling_optimized' 耗时：0.0011秒
Function 'double_linear_scaling' 耗时：0.4511秒
Function 'double_linear' 耗时：0.9625秒
